In [1]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('./final_data.csv')
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text,Id.1,Category
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,summarize: i have bought several of the vitali...,1.0,Pet Food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,summarize: product arrived labeled as jumbo sa...,2.0,Snacks
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,summarize: this is a confection that has been ...,3.0,Confectionery
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,summarize: if you are looking for the secret i...,4.0,Beverages
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,summarize: great taffy at a great price. there...,5.0,Pet Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...,summarize: we switched from the advance simila...,NaN,product review
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...","summarize: like the bad reviews say, the organ...",NaN,product review
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...,summarize: i wanted to solely breastfeed but w...,NaN,product review
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...,summarize: i love the fact that i can get this...,NaN,product review


In [5]:
data = df[:1000]
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text,Id.1,Category
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,summarize: i have bought several of the vitali...,1.0,Pet Food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,summarize: product arrived labeled as jumbo sa...,2.0,Snacks
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,summarize: this is a confection that has been ...,3.0,Confectionery
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,summarize: if you are looking for the secret i...,4.0,Beverages
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,summarize: great taffy at a great price. there...,5.0,Pet Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...,summarize: we switched from the advance simila...,NaN,product review
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...","summarize: like the bad reviews say, the organ...",NaN,product review
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...,summarize: i wanted to solely breastfeed but w...,NaN,product review
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...,summarize: i love the fact that i can get this...,NaN,product review


In [7]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained("./Finetuned_T5/tokenizer")
model = T5ForConditionalGeneration.from_pretrained("./Finetuned_T5/model").to(device)

In [9]:
rouge = evaluate.load("rouge")

In [10]:
def preprocess_data(row, tokenizer, max_input_length=512, max_target_length=50):
    """
    Tokenize a single row for T5 training.
    """
    inputs = row["preprocessed_text"] + "Category:" + row["Category"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    
    labels = tokenizer(row["Summary"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [11]:
dataset_train.reset_index(drop=True, inplace=True)
dataset_test.reset_index(drop=True, inplace=True)

In [12]:
tokenized_train = dataset_train.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

tokenized_test = dataset_test.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

In [13]:
tokenized_train

0      [input_ids, attention_mask, labels]
1      [input_ids, attention_mask, labels]
2      [input_ids, attention_mask, labels]
3      [input_ids, attention_mask, labels]
4      [input_ids, attention_mask, labels]
                      ...                 
795    [input_ids, attention_mask, labels]
796    [input_ids, attention_mask, labels]
797    [input_ids, attention_mask, labels]
798    [input_ids, attention_mask, labels]
799    [input_ids, attention_mask, labels]
Length: 800, dtype: object

In [ ]:
from transformers import Trainer, TrainingArguments

#Fine Tuning the model
training_args = TrainingArguments(
    output_dir="experiments",  
    evaluation_strategy="steps",    
    eval_steps=100,                
    logging_dir="./logs",           
    logging_steps=50,             
    save_steps=50,                
    save_total_limit=5,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    num_train_epochs=10,             
    learning_rate=5e-5,             
    weight_decay=0.01,             
    warmup_steps=50,               
    fp16=torch.cuda.is_available(), 
    report_to="none"              
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)


C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\shiva\AppData\Local\Temp\ipykernel_8476\1971552976.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
history = trainer.train()

In [37]:
def summarize_text_finetuned(text, max_input_length=512, max_output_length=15):
    """
    Summarizes input text using a T5 model.
    """
    input_text = text.strip()
    
    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to(device)
    
    # Generate the summary
    outputs = trainer.model.generate(inputs, max_length=max_output_length, min_length=2, length_penalty=2.0, num_beams=4)
    
    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


In [38]:
dataset_test["generated_summary"] = dataset_test["preprocessed_text"].apply(summarize_text_finetuned)

In [39]:
dataset_test["generated_summary"]

0                                        cheesy & cheesy
1      I love the ready to spread frosting, especiall...
2        I don't think it would be the right consistency
3       Tastes better than other low glycemic sweeteners
4                                   I love this product.
                             ...                        
195             a great fish - perfect for all purposes!
196    Great for school snacks, lunch boxes, and scho...
197    Not a dragon breath person, but a good breath ...
198    Good for a Natural Obsession - Good for a Natural
199             Great taste, but the berries had melted.
Name: generated_summary, Length: 200, dtype: object

## Evaluation

In [40]:
results = rouge.compute(
    predictions=dataset_test["generated_summary"].tolist(),
    references=dataset_test["Summary"].tolist()
)

In [41]:
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1770
rouge2: 0.0749
rougeL: 0.1685
rougeLsum: 0.1693


In [22]:
d = df["preprocessed_text"][5000:5200].apply(summarize_text_finetuned)

In [23]:
results = rouge.compute(
    predictions=d.tolist(),
    references=df["Summary"][5000:5200].tolist()
)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1135
rouge2: 0.0306
rougeL: 0.1108
rougeLsum: 0.1108


In [27]:
from bert_score import score

P, R, F1 = score(d.tolist(), df["Summary"][5000:5200].tolist(), lang='en', verbose=True)

print(F1.mean())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 7/7 [00:04<00:00,  1.52it/s]


computing greedy matching.


100%|██████████| 4/4 [00:00<00:00, 30.37it/s]

done in 4.76 seconds, 42.03 sentences/sec
tensor(0.8443)
